<p><font size="6"><b>Automate data validation - describe data</b></font></p>

> *© 2021, Joris Van den Bossche and Stijn Van Hoey  (<mailto:jorisvandenbossche@gmail.com>, <mailto:stijnvanhoey@gmail.com>). Licensed under [CC BY 4.0 Creative Commons](http://creativecommons.org/licenses/by/4.0/)*

---

## Introduction

When running the same analysis (code) on newly incoming data from which the data is _expected to have_ the same characteristics (format, data type,...) can become tedious aserrors occur due to a unexpected changes in the data.

Creating quarterly reports, processing data from a repeated experiment, comparing scenario's... are just a few examples of repeated analysis on data. To overcome manual checks of the data quality, setting up an __automated validation__ can be worthwhile. In this notebook, some frameworks targeted on Pandas DataFrames are highlighted:

- [Great expectations](https://greatexpectations.io/)
- [Pandera](https://pandera.readthedocs.io)
- [Frictionless data](https://frictionlessdata.io/)

__Note__ Imports of the packages are grouped per section/framework

## Great expectations

In [1]:
import json
from ruamel import yaml

import pandas as pd

import great_expectations as ge
from great_expectations import DataContext
from great_expectations.core import ExpectationSuite
from great_expectations.core.batch import RuntimeBatchRequest, BatchRequest
from great_expectations.validator.validator import Validator
from great_expectations.checkpoint.checkpoint import SimpleCheckpoint

[Great expectations](https://greatexpectations.io/) provides a entire framework for data quality check, aka 'production-ready' data validation: connecting to different data sources, notifications, automated triggers,... This provides a powerful ecosystem when working with continuous incoming data in a corporate environment, but the large set of functionalities can be overwhelming. 

In this introduction the focus is on setting up a set of _expectations_ (data validation rules) and applying these rules on a Pandas DataFrame data set loaded in memory.

### Start using great expectations

_Great expectations_ requires a specific folder-structure, which can be setup using the `init` command:

```
great_expectations init
```

See also the [getting started documentation](https://docs.greatexpectations.io/docs/tutorials/getting_started/tutorial_overview).

### Define _expectations_ within Jupyter notebook interactively

Defining a set of _expectations_ can be done using an existing data set. Let's start from the casualties data set used earlier:

In [31]:
casualties_raw = pd.read_csv("./data/TF_ACCIDENTS_VICTIMS_2020.zip", 
                         compression='zip', 
                         sep="|", 
                         low_memory=False)
casualties_raw.head()

,DT_DAY,DT_HOUR,CD_DAY_OF_WEEK,TX_DAY_OF_WEEK_DESCR_FR,TX_DAY_OF_WEEK_DESCR_NL,MS_VICT,MS_VIC_OK,MS_SLY_INJ,MS_SERLY_INJ,MS_DEAD_30_DAYS,...,TX_ADM_DSTR_DESCR_NL,CD_PROV_REFNIS,TX_PROV_DESCR_FR,TX_PROV_DESCR_NL,CD_RGN_REFNIS,TX_RGN_DESCR_FR,TX_RGN_DESCR_NL,CD_SEX,TX_SEX_DESCR_FR,TX_SEX_DESCR_NL
0,2020-09-24,15,4,Jeudi,donderdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,2,Féminin,Vrouwelijk
1,2020-10-25,14,7,Dimanche,zondag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
2,2020-09-24,15,4,Jeudi,donderdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
3,2020-12-01,15,2,Mardi,dinsdag,1,1,0,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
4,2020-12-16,17,3,Mercredi,woensdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk


Different options are provided to create new expectations (_great expectations_ provides automatically generated notebook or have a look at the [automated _profilers_](https://docs.greatexpectations.io/docs/guides/expectations/advanced/how_to_create_a_new_expectation_suite_using_rule_based_profilers) as well). 

The following approach starts from our Pandas DataFrame, connecting it to _great expectations_ and adding new rules interactively in this session:

In [32]:
# https://greatexpectations.io/expectations/
my_df = ge.from_pandas(casualties_raw)

For each rule defined in _great expectations_, the `my_df` variable tracks the set of rules defined and run (duplicate runs are ignored):

_check some rule options yourself using `my_df.expect_` + TAB-button_

In [33]:
my_df.expect_column_distinct_values_to_be_in_set(column="TX_SEX_DESCR_NL",
                                                 value_set=["Vrouwelijk", "Mannelijk"])

{
  "meta": {},
  "result": {
    "observed_value": [
      "Mannelijk",
      "Onbekend",
      "Vrouwelijk"
    ],
    "element_count": 66130,
    "missing_count": null,
    "missing_percent": null
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false
}

In [34]:
my_df.expect_column_min_to_be_between(column="MS_VICT", min_value=0)

{
  "meta": {},
  "result": {
    "observed_value": 1,
    "element_count": 66130,
    "missing_count": null,
    "missing_percent": null
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true
}

In [35]:
my_df.expect_column_values_to_not_be_null(column="DT_DAY")

{
  "meta": {},
  "result": {
    "element_count": 66130,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true
}

Let's check the _expectations_ defined up to this point:

In [36]:
my_df.get_expectation_suite(discard_failed_expectations=False)

{
  "meta": {
    "great_expectations_version": "0.14.10"
  },
  "expectation_suite_name": "default",
  "ge_cloud_id": null,
  "expectations": [
    {
      "meta": {},
      "expectation_type": "expect_column_distinct_values_to_be_in_set",
      "kwargs": {
        "column": "TX_SEX_DESCR_NL",
        "value_set": [
          "Vrouwelijk",
          "Mannelijk"
        ]
      }
    },
    {
      "meta": {},
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "MS_VICT",
        "min_value": 0
      }
    },
    {
      "meta": {},
      "expectation_type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "DT_DAY"
      }
    }
  ],
  "data_asset_type": "Dataset"
}

Store them to reuse the set of expectations later on. We store the `json` output in the `expectations` subfolder of within the folder structure. The name of the file, i.e. `be_casualties` will be used later on to use this set of expectations:

In [37]:
with open("./great_expectations/expectations/be_casualties.json", "w") as my_file:    
    my_file.write(json.dumps(my_df.get_expectation_suite(discard_failed_expectations=False).to_json_dict()) )

### Setup to run great expectations framework on in-memory DataFrame

These steps only need to be done __once for a new project__. The aim is to create the necessary configuiration to apply a set of _expectations_ on a _in memory_ Pandas DataFrame.

1. First let `great expectations` get the context from all the configuration in the different subfolders: 

In [38]:
context = ge.get_context()

_Note: this also loads the `be_casualties` expectations we already defined in the previous step._

2. Define an in-memory Pandas 'data source'

The framework provides a [wider range of connectors](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_choose_which_dataconnector_to_use) (e.g. csv-file, database, cloud storage,...). This example provides the configuration to run it within the context of a Python session (Jupyter notebook) on an in-memory DataFrame:

In [39]:
datasource_yaml = f"""
name: pandas_casualties
class_name: Datasource
module_name: great_expectations.datasource
execution_engine:
  module_name: great_expectations.execution_engine
  class_name: PandasExecutionEngine
data_connectors:
    casualties_memory:
        class_name: RuntimeDataConnector
        batch_identifiers:
            - year
"""
context.add_datasource(**yaml.safe_load(datasource_yaml))

3. A _checkpoint_ links a data set with a set of _expectations_:

In [40]:
checkpoint_config = {
    "name": "casualties_check",
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": {
                "datasource_name": "pandas_casualties",
                "data_connector_name": "casualties_memory",
                "data_asset_name": "casualties",
            },
            "expectation_suite_name": "be_casualties"
        }
    ],
}
context.add_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "casualties_check",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "pandas_casualties",
        "data_connector_name": "casualties_memory",
        "data_asset_name": "casualties"
      },
      "expectation_suite_name": "be_casualties"
    }
  ]
}

This is the necessary configuration to be able to run the data validation. Check the subfolder [./great_expectations](./great_expectations) to see the created configuration files stored on disk.

### Apply a set of _expectations_ on a (new version of) data set

We have now all elements together to check a data set with the defined expectations on a (new) data set and check the created data validation report.

The code in this section is required to run an evaluation on __any new version of the data__.

In [41]:
casualties_raw = pd.read_csv("./data/TF_ACCIDENTS_VICTIMS_2020.zip", 
                         compression='zip', 
                         sep="|", 
                         low_memory=False)
casualties_raw.head()

,DT_DAY,DT_HOUR,CD_DAY_OF_WEEK,TX_DAY_OF_WEEK_DESCR_FR,TX_DAY_OF_WEEK_DESCR_NL,MS_VICT,MS_VIC_OK,MS_SLY_INJ,MS_SERLY_INJ,MS_DEAD_30_DAYS,...,TX_ADM_DSTR_DESCR_NL,CD_PROV_REFNIS,TX_PROV_DESCR_FR,TX_PROV_DESCR_NL,CD_RGN_REFNIS,TX_RGN_DESCR_FR,TX_RGN_DESCR_NL,CD_SEX,TX_SEX_DESCR_FR,TX_SEX_DESCR_NL
0,2020-09-24,15,4,Jeudi,donderdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,2,Féminin,Vrouwelijk
1,2020-10-25,14,7,Dimanche,zondag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
2,2020-09-24,15,4,Jeudi,donderdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
3,2020-12-01,15,2,Mardi,dinsdag,1,1,0,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
4,2020-12-16,17,3,Mercredi,woensdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk


In [42]:
context = ge.get_context()

We can now _run_ a checkpoint on our DataFrame:

In [43]:
results = context.run_checkpoint(
    checkpoint_name="casualties_check",
    batch_request={
        "runtime_parameters": {"batch_data": casualties_raw},
        "batch_identifiers": {
            "year": 2020
        },
    },
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

See the _expectation_ result in an interactive page:

In [45]:
context.build_data_docs()
context.open_data_docs()

<div class="alert alert-info">

**Great expectations**

- An entire ecosystem with lots of integrations and configuration options. 
- Once a set of _expectations_ is setup and the Context/DataSource/Checkpoint has been setup, a very comprehensive __data validation report__ is provided.

</div>

## Pandera

[Pandera](https://pandera.readthedocs.io) provides a similar functionality as Great expectations, but requires less configuration setup:

In [46]:
import pandas as pd
import pandera as pa

In [47]:
casualties_raw = pd.read_csv("./data/TF_ACCIDENTS_VICTIMS_2020.zip", 
                         compression='zip', 
                         sep="|", 
                         low_memory=False,
                         parse_dates=["DT_DAY"])
casualties_raw.head()

,DT_DAY,DT_HOUR,CD_DAY_OF_WEEK,TX_DAY_OF_WEEK_DESCR_FR,TX_DAY_OF_WEEK_DESCR_NL,MS_VICT,MS_VIC_OK,MS_SLY_INJ,MS_SERLY_INJ,MS_DEAD_30_DAYS,...,TX_ADM_DSTR_DESCR_NL,CD_PROV_REFNIS,TX_PROV_DESCR_FR,TX_PROV_DESCR_NL,CD_RGN_REFNIS,TX_RGN_DESCR_FR,TX_RGN_DESCR_NL,CD_SEX,TX_SEX_DESCR_FR,TX_SEX_DESCR_NL
0,2020-09-24,15,4,Jeudi,donderdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,2,Féminin,Vrouwelijk
1,2020-10-25,14,7,Dimanche,zondag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
2,2020-09-24,15,4,Jeudi,donderdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
3,2020-12-01,15,2,Mardi,dinsdag,1,1,0,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk
4,2020-12-16,17,3,Mercredi,woensdag,1,0,1,0,0,...,Arrondissement Antwerpen,10000,Province d’Anvers,Provincie Antwerpen,2000,Région flamande,Vlaams Gewest,1,Masculin,Mannelijk


Define a set of rules as a `DataFrameSchema`:

In [48]:
# define schema
schema = pa.DataFrameSchema({
    "MS_VICT": pa.Column(int, checks=[
        pa.Check.greater_than_or_equal_to(0)
    ]),
    "DT_DAY": pa.Column("datetime64"),
    "TX_SEX_DESCR_NL": pa.Column(
        str, 
        checks=pa.Check.isin(["Mannelijk", "Vrouwelijk"])
    ),
    "CD_.+": pa.Column(
        int,
        regex=True
    ),    
})

Apply a schema to a DataFrame:

In [81]:
# validated_df = schema(casualties_raw, lazy=True)  # RUN to see report

__Note__ Use the `lazy=True` option to see all errors in a single Exception.

Integrate it with existing code and packages using the `check_input` decorator. This can overcome repeated checks at the start of your data processing functions.

In [50]:
from pandera import check_input

# by default, `check_input` assumes that the first argument is dataframe/series.
@check_input(schema)
def dataprocessor(df):
    """My data analysis functionality...
    # ...
    """
    return df

#dataprocessor(casualties_raw)

This overcomes repeated checks at the start of a processing function, e.g.

```python
def dataprocessor(df):
    """My data analysis functionality...
    # ...
    """
    if "name" not in df.columns:
        raise Exception("...")
    # ...
    return df
```

<div class="alert alert-info">

**Pandera**

- Easy to setup and __integrate with existing workflow/code__.
- No detailed reporting included (error message summary).
    
</div>

## Frictionless data

In [94]:
from pprint import pprint
from frictionless import validate

In [148]:
# Create a small subset of cleaned casualties to illustrate frictionless setup
#!head -n 100 ./data/casualties.csv > ./data/casualties_example.csv

[Frictionless data](https://framework.frictionlessdata.io/) provides a ata management framework for Python to describe, extract, __validate__, and transform tabular data.

It can be used both as a command-line utility as with Python.

In [92]:
!frictionless --help

Usage: frictionless [OPTIONS] COMMAND [ARGS]...

  Describe, extract, validate and transform tabular data.

Options:
  --version
  --install-completion  Install completion for the current shell.
  --show-completion     Show completion for the current shell, to copy it or
                        customize the installation.
  --help                Show this message and exit.

Commands:
  api        Start API server
  describe   Describe a data source.
  extract    Extract a data source.
  transform  Transform data using a provided pipeline.
  validate   Validate a data source.


In [93]:
# CLI examples
#!frictionless describe ./data/casualties_example.csv > casualties_example.resource.yaml
#!frictionless validate ./data/casualties_example.csv

In [145]:
from frictionless import describe, validate

resource = describe("./data/casualties_example.csv")  # create automated initial version

# Overwrite an data type
resource.schema.get_field("n_victims_ok").type = 'integer'

# Save the specification to a yaml-file
resource.to_yaml("casualties_example.resource.yaml");

The data `Schema` can be made part of a [Data package](https://framework.frictionlessdata.io/docs/guides/describing-data#describing-a-package) (i.e. a csv with metadata) to communicate on the data specification.

The Frictionless framework provides following ways to check the data consistency:

- [Constraints](https://specs.frictionlessdata.io/table-schema/#constraints) as part of the description configuration, e.g. minimum/maximum, required, unique, patter,...
- [Validation checks](https://framework.frictionlessdata.io/docs/guides/validation-checks/) included int he Framework which can be defined on validate

In [146]:
resource = describe("./data/casualties_example.csv")

# Add additional rules to the data set Schema:
resource.schema.get_field("n_victims_ok").type = 'integer'
resource.schema.get_field("n_victims_ok").constraints["minimum"] = 0
resource.schema.get_field("gender").constraints["pattern"] = "male|female"
resource.schema.get_field("gender").constraints["required"] = True

# Save the specification to a yaml-file
resource.to_yaml("casualties_example.resource.yaml");

In [147]:
report = validate("casualties_example.resource.yaml",
                  checks=[{"code": "table-dimensions",   # additional validation check
                           "minFields": 10, 
                           "maxRows": 200}])
#report["tasks"][0]["errors"] # uncomment to see report

<div class="alert alert-info">

**Frictionless data**

- Targets csv (excel, json and sql) files, not specific to Pandas DataFrame.
- Provides a set of tools to exchange data with appropriate documentation (metadata). For example, it is used to share [camera trap data](https://tdwg.github.io/camtrap-dp/) in a structured and standardized format.

</div>

## Conclusion

Different tools exist in the Python landscape to validate data sets. Each of the frameworks have their own strengths and use cases:

<div class="alert alert-info">

**Conclusion**

- __Great Expectations__: When external data integration (databases, cloud storage,...) or _advanced reporting_ (e.g. to provide detailed/automated feedback) is essential.
- __Pandera__: Ideal for _personal_ (or small team) usage when doing data analysis in Pandas. _Minimal effort_ to get started.
- __Frictionless data__: Provides the tools to _share_ data in a documented and well-structured workflow, while keeping technical burden low. Does not expect collaborators to use Pandas (e.g. [frictionless-r](https://github.com/frictionlessdata/frictionless-r) for R users).
    
__Note:__ 

- Each framework supports the extension with _custom_ or _user-defined_ rules.
- Pandera can convert and use a [Frictionless data schema](https://pandera.readthedocs.io/en/stable/frictionless.html#frictionless-integration).
    
</div>